NGS data cleaning template  

In [1]:
## pip install chardet - to detect file encoding
# Detecting the encoding of a CSV file using chardet
import chardet

with open('ngs.csv', 'rb') as f:
    result = chardet.detect(f.read(100000))
    print(result)  # Output: {'encoding': 'Windows-1252', 'confidence': 0.99, 'language': ''}


{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


Step 1: Remove unuseful rows and add 'Effective Date' and 'Fund Name' columns

In [2]:
# iporting the dataset and extract effective date
import pandas as pd

df_raw = pd.read_csv('ngs.csv',encoding='cp1252',header = None)  # Use the detected encoding

# Extract effective date from the first cell
first_cell = df_raw.iloc[0, 0]
import re
match = re.search(r'\d{4}-\d{2}-\d{2}', str(first_cell))
effective_date = match.group(0) if match else None

print(f"Effective Date: {effective_date}")

Effective Date: 2024-12-31


In [3]:
# Skip the first row which contains the effective date
df = pd.read_csv('ngs.csv', encoding='cp1252',skiprows=1)  

# Add 'Effective Date' and 'Fund Name' columns
df['Effective Date'] = effective_date
df['Fund Name'] = 'NGS'
df['Option Name'] = 'Balance Growth'
print(df.head())

  ASSET CLASS INTERNALLY MANAGED OR EXTERNALLY MANAGED  \
0        CASH                                        -   
1        CASH                                        -   
2        CASH                                        -   
3        CASH                                        -   
4        CASH                                        -   

           NAME OF INSTITUTION NAME OF ISSUER / COUNTERPARTY  \
0         MACQUARIE GROUP LTD.                             -   
1  STATE STREET BANK AND TRUST                             -   
2  STATE STREET BANK AND TRUST                             -   
3                       UBS AG                             -   
4  STATE STREET BANK AND TRUST                             -   

  NAME OF FUND MANAGER NAME / KIND OF INVESTMENT ITEM CURRENCY  \
0                    -                              -      AUD   
1                    -                              -      AUD   
2                    -                              -      USD   
3 

Step 2: Find the cut-off point and remove unnecessary information at the end of the table

In [4]:
# find the cut-off point 
first_col = df.columns[0] 
cutoff_index = df[
    df[first_col].astype(str).str.contains(
        r"The value \(AUD\) and weighting \(%\) sub totals may not sum to 100%", na=False
    )
].index.min()
print("Cut main table before row:", cutoff_index)


Cut main table before row: 1947


In [5]:
# separate the main table and summary table
df_main = df.loc[:cutoff_index - 1]   # all rows before cutoff
df_summary = df.loc[cutoff_index:]    # cutoff row and everything after


In [ ]:
df_main.info()

Step 3: Merge and Rename columns

In [ ]:
# Rename some columns
df_main.rename(columns={
    df_main.columns[0]: 'Asset Class Name',
    df_main.columns[1]: 'Int/Ext',
    '% OWNERSHIP / PROPERTY HELD': '% Ownership'
}, inplace=True)

# Convert 'Int/Ext' to binary
def convert_int_ext(value):
    if isinstance(value, str):
        val = value.strip().upper()
        if val == 'INTERNALLY':
            return 0
        elif val in ['EXTERNALLY', '-']:
            return 1
    return ''  # leave blank if not matched

df_main['Int/Ext'] = df_main['Int/Ext'].apply(convert_int_ext)

# Combine 4 overlapping columns into one
df_main['Name/Kind of Investment Item'] = df_main[
    ['NAME OF INSTITUTION', 
     'NAME OF ISSUER / COUNTERPARTY', 
     'NAME OF FUND MANAGER', 
     'NAME / KIND OF INVESTMENT ITEM']
].bfill(axis=1).iloc[:, 0]

# Keep Currency column as it is

# Split 'SECURITY IDENTIFIER' into 2 columns by space delimeter
df_main[['Stock ID', 'Listed Country']] = df_main['SECURITY IDENTIFIER'].str.extract(r'^([^ ]+)\s+(.*)$')


# Final selection and reordering of columns
df_main = df_main[[
    'Effective Date',
    'Fund Name',
    'Option Name',
    'Asset Class Name',
    'Int/Ext',
    'Name/Kind of Investment Item',
    'CURRENCY',
    'Stock ID',
    'Listed Country',
    '% Ownership',
    'UNITS HELD',
    'ADDRESS',
    'VALUE(AUD)',
    'WEIGHTING(%)'
]]


Step 4: Deal with 'Sub total' items and standardise column dtype


In [ ]:
## Deal with Sub total items 
# Define mask to detect rows with 'sub total' in 'Asset Class Name'
mask_subtotal = df_main['Asset Class Name'].str.contains(r'sub\s*total', case=False, na=False)

# Move 'Asset Class Name' value to 'Name/Kind of Investment Item' in those rows
df_main.loc[mask_subtotal, 'Name/Kind of Investment Item'] = df_main.loc[mask_subtotal, 'Asset Class Name']

# Clear 'Asset Class Name' in those rows
df_main.loc[mask_subtotal, 'Asset Class Name'] = ''


In [13]:
## Change dtype of 'Value(AUD) and 'WEIGHTING(%)' to float

# Value(AUD) — remove '$' and ',' and convert to float
df_main['VALUE(AUD)'] = (
    df_main['VALUE(AUD)']
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .replace('', pd.NA)
    .astype(float)
)

# Weighting(%) — remove '%' and convert to float
df_main['WEIGHTING(%)'] = (
    df_main['WEIGHTING(%)']
    .astype(str)
    .str.replace('%', '', regex=False)
    .replace('', pd.NA)
    .astype(float)
)

print(df_main[['VALUE(AUD)','WEIGHTING(%)']].info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   VALUE(AUD)    1940 non-null   float64
 1   WEIGHTING(%)  1940 non-null   float64
dtypes: float64(2)
memory usage: 30.5 KB
None
